In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler

from sklearn.model_selection import train_test_split

from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import TextClassificationPipeline

import tensorflow as tf
import pandas as pd
import json
import gc

import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopw = stopwords.words('english')

import seaborn as sns
import matplotlib.pyplot as plt
from plotly.offline import iplot

from tqdm import tqdm




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# 1. Processing and Cleaning

In [3]:
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/article/articles.csv',encoding='latin1')
df1.head()

,Id,Heading,Article.Banner.Image,Outlets,Article.Description,Full_Article,Article_Type,Tonality
0,d6995462-5e87-453b-b64d-e9f1df6e94d2,"A Puzzling Maneuver, Then Freefall: NTSB Repor...",NaN,Essex Caller,<p>The helicopter that crashed in Southeast Al...,<p>The helicopter that crashed in Southeast Al...,Commercial,Negative
1,8b05e939-a89e-4548-b92b-013822e8ee7d,Bells Nexus Air Taxi Concept Rings Changes Fo...,NaN,Aviation Week Network,<p>A year after teasing the fledgling electric...,<p>A year after teasing the fledgling electric...,Commercial,Positive
2,69fcd400-bceb-4255-8277-619f2d68ac0b,Bell Helicopter Show Air Taxi Nexus,http://images.tmtpost.com/uploads/images/2019/...,TMTPost,<p>Bell released the full-size design of the v...,<p>Bell released the full-size design of the v...,Commercial,Positive
3,17943578-c11b-414b-b3f5-063d3a93157b,BELL DÉVOILE LA CONCEPTION INTÉGRALE DE SON TA...,http://www.fredzone.org/wp-content/uploads/201...,Fredzone,<p>Bell est une soci&eacute;t&eacute; am&eacut...,<p>Bell est une soci&eacute;t&eacute; am&eacut...,Commercial,Positive
4,f33c7b11-5f77-4a98-bb2e-d36689042aea,Les premiers retours dOlivier Ezratty,NaN,FrenchWeb,<p>It was still anecdotal to observe the explo...,<p>It was still anecdotal to observe the explo...,Commercial,Positive


In [4]:
df = df1[['Heading', 'Article.Description', 'Full_Article', 'Tonality', 'Outlets','Article_Type']]
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4305 entries, 0 to 4304
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Heading              4305 non-null   object
 1   Article.Description  4305 non-null   object
 2   Full_Article         4305 non-null   object
 3   Tonality             3873 non-null   object
 4   Outlets              4305 non-null   object
 5   Article_Type         4305 non-null   object
dtypes: object(6)
memory usage: 201.9+ KB


In [5]:
missing_values = df.isna().sum()
missing_values


Heading                  0
Article.Description      0
Full_Article             0
Tonality               432
Outlets                  0
Article_Type             0
dtype: int64

In [6]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3873 entries, 0 to 4303
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Heading              3873 non-null   object
 1   Article.Description  3873 non-null   object
 2   Full_Article         3873 non-null   object
 3   Tonality             3873 non-null   object
 4   Outlets              3873 non-null   object
 5   Article_Type         3873 non-null   object
dtypes: object(6)
memory usage: 211.8+ KB


In [7]:
X = df[['Heading', 'Article.Description', 'Full_Article', 'Tonality', 'Outlets']]
y = df['Article_Type']

In [8]:
y.unique()

array(['Commercial', 'Military', 'Training', 'Others', 'Financing',
       'Support & Services', 'Executives'], dtype=object)

In [9]:
label_encoder = LabelEncoder()
df['Article_Type'] = label_encoder.fit_transform(df['Article_Type'])


<ipython-input-9-d53178161046>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [10]:
import re

def clean_text(text):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', text)
    clean_text = re.sub(r'[^a-zA-Z0-9\s]', '', cleantext)

    return clean_text.strip()

In [11]:
df['Full_Article'] = df['Full_Article'].apply(lambda x: clean_text(x))
df['Article.Description'] = df['Article.Description'].apply(lambda x: clean_text(str(x)))
df['Heading'] = df['Heading'].apply(lambda x: clean_text(str(x)))
df.head()


,Heading,Article.Description,Full_Article,Tonality,Outlets,Article_Type
0,A Puzzling Maneuver Then Freefall NTSB Report ...,The helicopter that crashed in Southeast Alask...,The helicopter that crashed in Southeast Alask...,Negative,Essex Caller,0
1,Bells Nexus Air Taxi Concept Rings Changes For...,A year after teasing the fledgling electric ve...,A year after teasing the fledgling electric ve...,Positive,Aviation Week Network,0
2,Bell Helicopter Show Air Taxi Nexus,Bell released the fullsize design of the verti...,Bell released the fullsize design of the verti...,Positive,TMTPost,0
3,BELL DVOILE LA CONCEPTION INTGRALE DE SON TAXI...,Bell est une socieacuteteacute ameacutericaine...,Bell est une socieacuteteacute ameacutericaine...,Positive,Fredzone,0
4,Les premiers retours dOlivier Ezratty,It was still anecdotal to observe the explosio...,It was still anecdotal to observe the explosio...,Positive,FrenchWeb,0


In [12]:
df['Article_Type'].value_counts()

Article_Type
0    2235
3    1492
1      59
4      52
5      26
2       7
6       2
Name: count, dtype: int64

In [13]:
X = df.drop('Article_Type', axis=1)
y = df['Article_Type']

oversampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X, y)

df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
df_resampled['Article_Type'] = y_resampled


print(df_resampled['Article_Type'].value_counts())


Article_Type
0    2235
3    2235
6    2235
4    2235
2    2235
5    2235
1    2235
Name: count, dtype: int64


In [14]:
data_texts = df_resampled['Full_Article'].to_list()
data_labels = df_resampled['Article.Description'].to_list()

# Train Test Split

In [15]:
train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size = 0.2, random_state = 0 )


train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size = 0.01, random_state = 0 )

In [16]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_texts, truncation = True, padding = True  )

val_encodings = tokenizer(val_texts, truncation = True, padding = True )

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning:

`resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.



config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [17]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))


val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

print("train_dataset and val_dataset ready")


train_dataset and val_dataset ready


## Model Creation


In [18]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08)

model.compile(optimizer=optimizer,
              loss=model.hf_compute_loss,
              metrics=['accuracy'])

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [19]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.fit(train_dataset.shuffle(1000).batch(16),
          epochs=5,
          batch_size=16,
          validation_data=val_dataset.shuffle(1000).batch(16),
          callbacks=[early_stopping])

# Display the model's architecture
model.summary()

Epoch 1/5


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


UnimplementedError: Graph execution error:

Detected at node Cast_1 defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/python3.10/dist-packages